In [2]:
import pandas as pd
import numpy as np

In [3]:
# prompt: connect Google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
Q_df = pd.read_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/test.csv")

In [5]:
A_gemini_df = pd.read_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/processed_financial_analysis_flash2-5_01.csv")

In [6]:
A_gemini_df.domain.unique()

array(['out of domain', 'ethical finance', 'economics', 'stock movement',
       'marketing', 'finance management', 'accounting',
       'financial reporting', 'fixed income'], dtype=object)

In [7]:
A_gemini_df.answer.unique()

array(['D', 'B', 'A', 'Rise', 'C', 'Fall'], dtype=object)

In [8]:
Q_df['query'].iloc[1]

'Answer the question with the appropriate options A, B, C and D. Please respond with the exact answer A, B, C or D only. Do not be verbose or provide extra information. \nQuestion: Who of these is the entrepreneur?\nAnswer Choices: A: Barack Obama, B: James Dyson, C: Damien Hirst, D: Mo Farah \nAnswer:'

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
# prepare the model input
prompt = "Hello, how are you?"
messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)

model_inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **model_inputs,
    max_new_tokens=256,
)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
assistant_output = response.split("</think>")[-1].strip()
print(assistant_output)

Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with anything you need! 😊 How can I assist you today?


In [17]:
idx = 0
query = Q_df['query'].iloc[idx]

system_message = """
You are a financial expert. Your job is to answer the questions of given contents strictly, and answer only choices given (choice's description excluded)

Your answer must be structured as:
Answer: <A/B/C/D/E/Fall/Rise>
"""

In [27]:
conversation = [
    {"role": "system", "content": system_message},
    {"role": "user",   "content": query},
]

print(tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True, enable_thinking=True))

<|im_start|>system

You are a financial expert. Your job is to answer the questions of given contents strictly, and answer only choices given (choice's description excluded)

Your answer must be structured as:
Answer: <A/B/C/D/E/Fall/Rise>
<|im_end|>
<|im_start|>user
ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม

คำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment' ที่พนักงานได้รับขอบเขตที่ใหญ่ขึ้นในการตัดสินใจว่าจะจัดระเบียบงานของตนอย่างไร หรือ 'job enlargement' ที่พนักงานได้รับมอบหมายงานที่หลากหลายมากขึ้น

ตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing

คำตอบ:<|im_end|>
<|im_start|>assistant



In [29]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user",   "content": query},
  ]

full_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True,
  )

inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=256*8, do_sample=True, temperature=0.01, top_p=0.9, top_k=1)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Only show the assistant's reply (strip the system/user)
assistant_output = response.split("</think>")[-1].strip()
print(assistant_output)

Answer: D


In [30]:
assistant_output.split("Answer: ")[-1].split("\n")[0]

'D'

In [31]:
A_gemini_df.answer.iloc[idx]

'D'

In [33]:
len(Q_df)

499

In [34]:
tokens = tokenizer.encode(query, add_special_tokens=False)
print(f"🔢 Number of tokens: {len(tokens)}")

🔢 Number of tokens: 225


### create reusable function

In [36]:
def extract_answer_from_model(system_message, prompt, tokenizer, model, AnsKey="Answer: "):
    # Construct chat format
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]

    # Convert to full prompt string
    full_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True,
    )

    # Tokenize and run generation
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256*8,
        do_sample=True,
        temperature=0.01,
        top_p=0.9,
        top_k=1
    )

    # Decode output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract assistant reply (after </think>, if present)
    assistant_output = response.split("</think>")[-1].strip()

    try:
        answer = assistant_output.split(AnsKey, 1)[-1].split("\n")[0].strip()
    except Exception:
        answer = "?"

    return answer

In [37]:
extract_answer_from_model(system_message, query, tokenizer, model)

'D'

### Loop Agents

In [38]:
from tqdm import tqdm

ans_train = []
gt_train = []

for idx in range(len(Q_df)):
    prompt = Q_df['query'][idx]

    answer = extract_answer_from_model(system_message, prompt, tokenizer, model)

    print(f"sample : {idx+1}/{len(Q_df)}")
    print("Inputs :", prompt[:20])
    print("ground_truth :", A_gemini_df['answer'][idx])
    print("LLM          :", answer)
    print("----------------------------------------------------")

    ans_train.append(answer)
    gt_train.append(A_gemini_df['answer'][idx])

sample : 1/499
Inputs : ตอบคำถามด้วยตัวเลือก
ground_truth : D
LLM          : D
----------------------------------------------------
sample : 2/499
Inputs : Answer the question 
ground_truth : B
LLM          : B
----------------------------------------------------
sample : 3/499
Inputs : Answer the question 
ground_truth : A
LLM          : A
----------------------------------------------------
sample : 4/499
Inputs : ตอบคำถามด้วยตัวเลือก
ground_truth : A
LLM          : <A/B/C/D/E/Fall/Rise>
----------------------------------------------------
sample : 5/499
Inputs : Given the data and t
ground_truth : Rise
LLM          : Rise
----------------------------------------------------
sample : 6/499
Inputs : Answer the question 
ground_truth : C
LLM          : C
----------------------------------------------------
sample : 7/499
Inputs : Answer the question 
ground_truth : B
LLM          : B
----------------------------------------------------
sample : 8/499
Inputs : วิเคราะห์ข้อมูลและทว
groun

In [ ]:
#ans_train = [x.split(".")[0] for x in ans_train]

In [39]:
import numpy as np
np.unique(ans_train)

array(['<A/B/C/D/E/Fall/Rise>', 'A', 'B', 'C', 'D', 'Down', 'Fall',
       'Rise', 'ขึ้น', 'คำตอบ: C', 'คำตอบ: D'], dtype='<U21')

In [42]:
import numpy as np

# Normalization function
def normalize_answer(ans_):
    ans = ans_.strip().upper()

    if "<A/B/C/D/E/Fall/Rise>" in ans_:
        return ans_

    if 'A' in ans and 'FALL' not in ans:
        return 'A'
    elif 'B' in ans:
        return 'B'
    elif 'C' in ans:
        return 'C'
    elif 'D' in ans:
        return 'D'
    else:
        return ans_

# Apply normalization
normalized = np.array([normalize_answer(a) for a in ans_train])

# Show unique cleaned answers
cleaned = np.unique(normalized)
print(cleaned)

['<A/B/C/D/E/Fall/Rise>' 'A' 'B' 'C' 'D' 'Fall' 'Rise' 'ขึ้น']


In [43]:
ans_train_ = normalized

In [44]:
# Calculate accuracy
correct = sum(p == g for p, g in zip(ans_train_, ans_train))
accuracy = correct / len(gt_train)

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 99.20%


In [45]:
# Calculate accuracy
correct = sum(p == g for p, g in zip(ans_train_, gt_train))
accuracy = correct / len(gt_train)

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 75.35%


In [46]:
submission_df = pd.read_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/submission.csv")
submission_df

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,NaN
4,bcca13bc-2675-4645-82cc-7e4c412ed294,NaN
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,NaN
495,73c720b5-1101-4790-af52-3366823e1d32,NaN
496,357db18f-d872-416e-a07f-753099853d9c,NaN
497,2d8b1419-1c46-4e83-892a-081fb417de38,NaN


In [47]:
submission_df['answer'] = ans_train_
submission_df

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,<A/B/C/D/E/Fall/Rise>
4,bcca13bc-2675-4645-82cc-7e4c412ed294,Rise
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,B
495,73c720b5-1101-4790-af52-3366823e1d32,A
496,357db18f-d872-416e-a07f-753099853d9c,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,<A/B/C/D/E/Fall/Rise>


In [48]:
submission_df.to_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/submission/ubmission_Qwen_Qwen3_8B_think_raw.csv", index=False)

In [53]:
submission_mix1_df = submission_df.copy()

In [52]:
first_df = pd.read_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/submission/submission_Qwen3-8B_PpromptImp1.csv")

In [55]:
mask = submission_mix1_df.answer == "<A/B/C/D/E/Fall/Rise>"
submission_mix1_df.loc[mask] = first_df.loc[mask]

In [58]:
# Calculate accuracy
correct = sum(p == g for p, g in zip(submission_mix1_df.answer, gt_train))
accuracy = correct / len(gt_train)

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 82.36%


In [59]:
submission_mix1_df.to_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/submission/ubmission_Qwen_Qwen3_8B_think_mix1.csv", index=False)